## Assymetric Cryptography

In [34]:
import random
import math
import codecs
import base64

In [35]:
class Utils:
    def __init__(self):
        '''if not used then delete'''
    
    def hex_to_dec(hex):
        deci = int(hex, 16)
        return deci

    def dec_to_hex(dec):
        hexa = hex(dec)
        return hexa

    def hex_to_b64(hex):
        hex = hex[2:] if(len(hex[2:]) % 2 == 0) else '0' + hex[2:]
        b64 = codecs.encode(codecs.decode(hex, 'hex'), 'base64').decode()
        return b64
       
    def b64_to_hex(b64):
        hexa = base64.b64decode(b64).hex()
        return hexa
    
    def dec_to_b64(dec):
        b64 = Utils.hex_to_b64(Utils.dec_to_hex(dec))
        return b64

    def b64_to_dec(b64):
        deci = Utils.hex_to_dec(Utils.b64_to_hex(b64))
        return deci

    def is_square(i):
        return i == math.isqrt(i) ** 2

    def mod_sqrt(n, p):

        points = []
        n = n % p
        for x in range (1, p):
            if ((x ** 2) % p == n) :
                points.append(x)
        return points

        

In [36]:
class ElGamal:
    def __init__(self):
        pass

    def create_key(self, p, g, x):
        y = pow(g, x, p)
        public_key = { 'y': y, 'g': g, 'p': p }
        private_key = { 'x': x, 'p': p }
        return public_key, private_key

    def encrypt(self, message, key, public_key):
        a = pow(public_key['g'], key, public_key['p'])
        b = (pow(public_key['y'], key, public_key['p']) * (message % public_key['p'])) % public_key['p']
        return { 'a': a, 'b': b }

    def decrypt(self, message, private_key):
        return ((message['b'] % private_key['p']) * pow(message['a'], (private_key['p'] - 1 - private_key['x']), private_key['p'])) % private_key['p']

In [84]:
elgamal = ElGamal()

public_key, private_key = elgamal.create_key(2357, 2, 1751)

print("Public Key\t: " + str(public_key))
print("Private Key\t: " + str(private_key))

encrypted = elgamal.encrypt(2035, 1520, public_key)
print("Ciphertext\t: " + str(encrypted))

decrypted = elgamal.decrypt(encrypted, private_key)
print("Plaintext\t: " + str(decrypted))

Public Key	: {'y': 1185, 'g': 2, 'p': 2357}
Private Key	: {'x': 1751, 'p': 2357}
Ciphertext	: {'a': 1430, 'b': 697}
Plaintext	: 2035


In [90]:
class ElGamalMachine:
    def __init__(self):
        self.elgamal = ElGamal()
    
    def is_prime(self, num, test_count=1000):
        if(num == 1):
            return False
        if(test_count >= num):
            test_count = num - 1
        for _ in range(test_count):
            val = random.randint(1, num - 1)
            if(pow(val, num-1, num) != 1):
                return False
        return True
    def create_random_prime(self, bit):
        found = False
        while not found:
            p = random.randint(2**(bit-1)+1, 2**bit-1)
            if self.is_prime(p):
                return p
    
    def create_key(self, bit):
        p = self.create_random_prime(bit)
        g = random.randint(1, p - 1)
        x = random.randint(1, p -2)

        public_key, private_key = self.elgamal.create_key(p, g, x)

        public_key['y'] = Utils.dec_to_b64(public_key['y'])
        public_key['g'] = Utils.dec_to_b64(public_key['g'])
        public_key['p'] = Utils.dec_to_b64(public_key['p'])

        private_key['x'] = Utils.dec_to_b64(private_key['x'])
        private_key['p'] = Utils.dec_to_b64(private_key['p'])

        return public_key, private_key

    def encrypt(self, message, public_key):
        y = Utils.b64_to_dec(public_key['y'])
        g = Utils.b64_to_dec(public_key['g'])
        p = Utils.b64_to_dec(public_key['p'])

        p_key = { 'y': y, 'g': g, 'p': p }

        encrypted = ''
        for m in message:
            k = random.randint(1, p-2)
            res = self.elgamal.encrypt(ord(m), k, p_key)
            a = Utils.dec_to_b64(res['a'])
            b = Utils.dec_to_b64(res['b'])
            encrypted += a + b
        return encrypted

    def decrypt(self, encrypted, private_key):
        x = Utils.b64_to_dec(private_key['x'])
        p = Utils.b64_to_dec(private_key['p'])

        p_key = { 'x': x, 'p': p }
        
        encrypted = encrypted.split('=\n')[:-1]
        encrypted_paired = []
        for i in range(0, len(encrypted), 2):
            a = Utils.b64_to_dec(encrypted[i] + '=\n')
            b = Utils.b64_to_dec(encrypted[i+1] + '=\n')
            encrypted_paired.append({ 'a': a, 'b': b })

        message = ''
        for m in encrypted_paired:
            message += chr(self.elgamal.decrypt(m, p_key))
        return message

In [93]:
machine = ElGamalMachine()

public_key, private_key = machine.create_key(256)

encrypted = machine.encrypt('Aku adalah = x +', public_key)
print(encrypted)
print(machine.decrypt(encrypted, private_key))

qpbeB5JMYKNmixIIh0nPt+mUaGTHVR8b1JZ/mRqvsYc=
x2TvpS5QETq9X05OK8QXxhPL1wnKi7ac1s8EtltFAYg=
fPMmKVq8XFDhg9/QJq3u6JcLPwbl6HshXENGqOrpKjk=
nPiga2BEZ+OvIcAHW1KtNjBHGpLadfQxDirv5V/qqlU=
sTqgvyr2E8gDmm87L8m4VQM+2s9qbsiIK2vRfD2Xv5E=
BSUVrHRnq5frQ2mblxLgGEjXyJHzdhu117qGsaGgIqs=
gRu/SLuLpL4zhob/Gkszrjh1B2tqL+wUJ9TIJkDavpA=
lFLoRV1aT0Hc7/tmZMYIZMyY+qDi6eueLlDo9ohMQtU=
fz4NVfq69GbRk7JzOPBko146E1BDGlR0RhX/6QkVnL4=
kF85nTuaUTHFxRVZoByzTrVDtnLxttrAepAVOj+o6mA=
xBHhSs/MpTKW94AO0yHYdppsKIMMmoRBkFSLm8puiEM=
ENy0fZ0IjlvOCD5DJB24qPLnXW+TqUaP8U6vbZEfpUk=
zCTfQLsNbAMYtOVMcWHj/Eg+LmNMVSHYYJeHebMYfUg=
Bw9YtN30iA18PhHV8A1SJbZwGT+yhd+y6Wbtl4xemiY=
ddOvlZEZLg4ROWGScVRb6Cno33TyL6csWA5OJkkpIkY=
L6pc/3ySczBVH4rODjFH9wS29o4Rk+K1ybc7CLfifE8=
OdhebLPvuVVV7wRk/G9uYwpT/gYKrKArU/+x/tJkoxc=
xA1gUcSngcJyZsrCoElxmxEwZECUe/+YIq5Tu8GljRE=
YmPD9SAOVguSGNts10santYu264f/rv6IiZFhbRX+Lk=
Hg7mrGxI/2S7FSMzALhxnr+gokLVkbV5AMnSyUkiTtk=
id6db1GaomGyBgbd7YVpYBJg4itI2a8ihYZRtikhjBA=
wXWt9Z7IPJ4jOB6fmF/QbYKrtmHGB65aX5KIs9PQvHk=
gkSqoMvhNk

In [ ]:
class EllipticPoint:
    def __init__(self, x=0, y=0, inf=False):
        self.x = x if not inf else math.inf
        self.y = y if not inf else math.inf
        self.inf = inf

    def get_x(self):
        return self.x
    
    def get_y(self):
        return self.y
    
    def is_infinity(self):
        return self.inf
    
    def mirror(self):
        return EllipticPoint(self.x, -self.y, self.inf)

    def __eq__(self, point):
        if(isinstance(point, EllipticPoint)):
            if(self.inf):
                return self.inf == point.inf
            else:
                return self.x == point.x and self.y == point.y
            
        return False

    def __str__(self):
        return "({0}, {1})".format(self.x, self.y)

In [ ]:
class EllipticCurve:
    def __init__(self, a, b, p):
        ''' y^2 = x^3 + ax + b '''
        self.a = a
        self.b = b
        self.p = p
        self.points = []
        if(self.is_singular()):
            print("Error: Curve is Singular!.")
    
    def is_singular(self):
        return((((4 * (self.a ** 3)) + (27 * (self.b ** 2))) % self.p) == 0)

    def get_points(self):
        points = [EllipticPoint(inf=True)]
        
        for x in range(self.p):
            y_square = x ** 3 + self.a * x + self.b

            for point in Utils.mod_sqrt(y_square % self.p, self.p):
                points.append(EllipticPoint(x, point))
        self.points = points 
    
    def get_y_square(self, x):
        y_square = x ** 3 + self.a * x + self.b

        for point in Utils.mod_sqrt(y_square % self.p, self.p):
            return(EllipticPoint(x, point))

    def point_addition(self, P, Q):
        xp = P.get_x(); yp = P.get_y();
        xq = Q.get_x(); yq = Q.get_y();

        m = ((yp-yq) * pow((xp-xq), -1, self.p)) % self.p

        xr = (m ** 2 - xp - xq) % self.p
        yr = (m * (xp - xr) - yp) % self.p
        return EllipticPoint(xr, yr)

    def point_substraction(self, P, Q):
        return self.point_addition(P, Q.mirror())

    def point_multiplication(self, P):
        
        xp = P.get_x(); yp = P.get_y();

        if(yp == 0):
            return EllipticPoint(inf=True)
        else:
            m = ((3 * (xp ** 2) + self.a) * pow((2 * yp), -1, self.p)) % self.p
            xr = (m ** 2 - 2 * xp) % self.p
            yr = (m * (xp - xr) - yp) % self.p
            return EllipticPoint(xr, yr)
        


    

In [ ]:
curve = EllipticCurve(0, 7, 97)
curve.get_points()

P = EllipticPoint(0, 5)
Q = EllipticPoint(3, 4)

print(curve.point_addition(P, Q))

print("N: " + str(len(curve.points)))

# for x in curve.points:
#     print(x)
# M = 11

# k = 20

# found = False
# i = 0
# while(not found):
#     x = M * k + i
    
#     i = i + 1


(51, 12)
N: 79


In [ ]:
# ALICE
# Kunci Privat = a
# Kunci Publik = PA = a . B

# BOB
# Kunci Privat = b
# Kunci Publik = PB = b . B

# Bilangan Acak k dalam interval [1, p-1]

# Ciphertext(PM) = PC = [(kB), (PM + kPB)]
# b . (kB) 


In [ ]:
curve = EllipticCurve(-1, 16, 29)
curve.get_points()

P = EllipticPoint(5, 7)
Q = EllipticPoint(1, 25)
R = EllipticPoint(6, 20)


print(curve.point_multiplication(P))
print(curve.point_addition(Q, R))


(28, 4)
(23, 26)
